# Model Denemeleri

### Önce HuggingFace'ten, hali hazırda tamamen türkçe veriler üzerinde eğitilmiş BerTurk modeli denenecek

In [1]:
from transformers import AutoTokenizer, BertForPreTraining
import torch

tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-cased")
model = BertForPreTraining.from_pretrained("dbmdz/bert-base-turkish-cased")

In [2]:
sd_hf = model.state_dict()

for k, v in sd_hf.items():
    print(k, v.shape)

bert.embeddings.word_embeddings.weight torch.Size([32000, 768])
bert.embeddings.position_embeddings.weight torch.Size([512, 768])
bert.embeddings.token_type_embeddings.weight torch.Size([2, 768])
bert.embeddings.LayerNorm.weight torch.Size([768])
bert.embeddings.LayerNorm.bias torch.Size([768])
bert.encoder.layer.0.attention.self.query.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.self.query.bias torch.Size([768])
bert.encoder.layer.0.attention.self.key.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.self.key.bias torch.Size([768])
bert.encoder.layer.0.attention.self.value.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.self.value.bias torch.Size([768])
bert.encoder.layer.0.attention.output.dense.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.output.dense.bias torch.Size([768])
bert.encoder.layer.0.attention.output.LayerNorm.weight torch.Size([768])
bert.encoder.layer.0.attention.output.LayerNorm.bias torch.Size([768])
bert.encoder

In [3]:
torch.equal(sd_hf["bert.embeddings.word_embeddings.weight"], sd_hf["cls.predictions.decoder.weight"]) # tying kontrolü

True

In [4]:
sd_hf["bert.embeddings.word_embeddings.weight"].data_ptr() == sd_hf["cls.predictions.decoder.weight"].data_ptr()  # tying kontrolü

True

In [5]:
torch.equal(sd_hf["cls.predictions.decoder.bias"], sd_hf["cls.predictions.bias"]) # tying kontrolü

True

In [6]:
sd_hf["cls.predictions.decoder.bias"].data_ptr() == sd_hf["cls.predictions.bias"].data_ptr()  # tying kontrolü

True

In [7]:
inputs = tokenizer("Selamlar efendim", return_tensors="pt")
outputs = model(**inputs)

In [8]:
outputs.prediction_logits.shape, outputs.seq_relationship_logits.shape

(torch.Size([1, 4, 32000]), torch.Size([1, 2]))

In [9]:
from transformers import pipeline

fill_masker = pipeline(task="fill-mask", model="dbmdz/bert-base-turkish-cased")
fill_masker("Merhaba [MASK] efendi nasıl gidiyor?")

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


[{'score': 0.05243540555238724,
  'token': 3944,
  'token_str': 'Mehmet',
  'sequence': 'Merhaba Mehmet efendi nasıl gidiyor?'},
 {'score': 0.03593067452311516,
  'token': 7709,
  'token_str': 'hanım',
  'sequence': 'Merhaba hanım efendi nasıl gidiyor?'},
 {'score': 0.03291669860482216,
  'token': 3024,
  'token_str': 'bey',
  'sequence': 'Merhaba bey efendi nasıl gidiyor?'},
 {'score': 0.031684163957834244,
  'token': 7983,
  'token_str': 'hoca',
  'sequence': 'Merhaba hoca efendi nasıl gidiyor?'},
 {'score': 0.030081957578659058,
  'token': 10997,
  'token_str': 'Salih',
  'sequence': 'Merhaba Salih efendi nasıl gidiyor?'}]

In [10]:
from model import BertForPreTraining, BertConfig

%load_ext autoreload
%autoreload 2

In [11]:
default_config = BertConfig()
custom_model = BertForPreTraining(default_config)

In [12]:
custom_sd = custom_model.state_dict()

assert len(custom_sd) == len(sd_hf)

for custom, hf in zip(custom_sd.items(), sd_hf.items()):
    k, v = custom
    k_hf, v_hf = hf
    assert k == k_hf, f"{k} : {k_hf}"
    assert v.shape == v_hf.shape, f"{k} : {v.shape} != {v_hf.shape}"
    print(k, v.shape, "--------", k_hf, v_hf.shape)

bert.embeddings.word_embeddings.weight torch.Size([32000, 768]) -------- bert.embeddings.word_embeddings.weight torch.Size([32000, 768])
bert.embeddings.position_embeddings.weight torch.Size([512, 768]) -------- bert.embeddings.position_embeddings.weight torch.Size([512, 768])
bert.embeddings.token_type_embeddings.weight torch.Size([2, 768]) -------- bert.embeddings.token_type_embeddings.weight torch.Size([2, 768])
bert.embeddings.LayerNorm.weight torch.Size([768]) -------- bert.embeddings.LayerNorm.weight torch.Size([768])
bert.embeddings.LayerNorm.bias torch.Size([768]) -------- bert.embeddings.LayerNorm.bias torch.Size([768])
bert.encoder.layer.0.attention.self.query.weight torch.Size([768, 768]) -------- bert.encoder.layer.0.attention.self.query.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.self.query.bias torch.Size([768]) -------- bert.encoder.layer.0.attention.self.query.bias torch.Size([768])
bert.encoder.layer.0.attention.self.key.weight torch.Size([768, 768]) -

In [13]:
model_from_hf = BertForPreTraining.from_pretrained()
mm = model_from_hf.state_dict()

loading weights from pretrained bert: dbmdz/bert-base-turkish-cased


In [15]:
from transformers import pipeline

fill_masker = pipeline(task="fill-mask", model="dbmdz/bert-base-turkish-cased")
fill_masker("Merhaba [MASK] efendi nasıl gidiyor?") 

[{'score': 0.05243540555238724,
  'token': 3944,
  'token_str': 'Mehmet',
  'sequence': 'Merhaba Mehmet efendi nasıl gidiyor?'},
 {'score': 0.03593067452311516,
  'token': 7709,
  'token_str': 'hanım',
  'sequence': 'Merhaba hanım efendi nasıl gidiyor?'},
 {'score': 0.03291669860482216,
  'token': 3024,
  'token_str': 'bey',
  'sequence': 'Merhaba bey efendi nasıl gidiyor?'},
 {'score': 0.031684163957834244,
  'token': 7983,
  'token_str': 'hoca',
  'sequence': 'Merhaba hoca efendi nasıl gidiyor?'},
 {'score': 0.030081957578659058,
  'token': 10997,
  'token_str': 'Salih',
  'sequence': 'Merhaba Salih efendi nasıl gidiyor?'}]

In [ ]:
# pipeline __init__ inputs: task, model, tokenizer (model ile tokenizer (vocab) uyumlu olmalı)
# pipeline __call__ inputs: task_mlm : ( str, list[str], sample'lar batch'lenebilmesi için pad ile doldur) ((sent1, sent2), list[list[sent1, sent2]], sample'lar batch'lenebilmeli), numpy array alabilir, attn mask/token type id gerekirse otomatik oluşturulmalı (bu otomatik oluşturmayı model'e bırakabiliriz!)
# pipeline: tokenizasyon -> model(input) -> logits -> topk sampling -> çıktı yukarıdaki gibi


In [ ]:
a = tuple("ali", "veli")
for x, y in a:
    

In [110]:
from bert_implementation_tr.data.data_aux import get_tokenizer

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [136]:
tokenizer = get_tokenizer()

In [74]:
encoding = tokenizer(["[CLS] bu bir [MASK] denemedir! [SEP] Bu da 2. cümledir! [SEP]", "İkinci sample hayırlı olsun ke..."], max_length=20, padding="max_length", truncation=True, return_tensors="pt", add_special_tokens=True)
encoding

{'input_ids': tensor([[    2,  1989,  1941,     4, 10453,  1990,     5,     3,  1989,  1964,
            22,    18, 10196,  1990,     5,     3,     0,     0,     0,     0],
        [ 4271, 18188,  1930,  9470,  1966,  8164,  6120, 32000,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}

In [88]:
encoding["attention_mask"], encoding["attention_mask"].size()

(tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]),
 torch.Size([2, 20]))

In [76]:
encoding["token_type_ids"] # bu durumda A ve B yi ayrı ayrı vermen gerekli!

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [81]:
tokenizer.decode(encoding["input_ids"][1])

'i̇kinci sample hayırlı olsun ke... [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [78]:
tokenizer.decode(encoding["input_ids"][0], skip_special_tokens=True)

'bu bir denemedir! bu da 2. cümledir!'

In [79]:
tokenizer.decode(encoding["input_ids"][1])

'i̇kinci sample hayırlı olsun ke... [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [80]:
tokenizer.decode(encoding["input_ids"][1], skip_special_tokens=True)

'i̇kinci sample hayırlı olsun ke...'

### Yapılacaklar: 

1) temiz res conn
2) layer norm yerini ayarla [bunlar from pretrain olmadığı durumda otomatik olmalı bir şekilde hallet flag vs'ler ile]
3) derinlik etkili inits (andrej)
4) ayrıca şu sdpa'ya da bak hatta onu da gene bir şekilde işe dahil et (en azından test et süre, mem vs açısından) {model utils'te flops vs hesaplama vardı ilerde ona da bakabilirsin}

In [85]:
a = ["bu bir denemedir! bu da 2. cümledir!", "asfkasng ksamflk ksjfmk "]
print(a)
if isinstance(a, str):
    a = "[CLS] " + a + " [SEP]"
else:
    for i, t in enumerate(a):
        a[i] = "[CLS] " + t + " [SEP]"
print(a)

['bu bir denemedir! bu da 2. cümledir!', 'asfkasng ksamflk ksjfmk ']
['[CLS] bu bir denemedir! bu da 2. cümledir! [SEP]', '[CLS] asfkasng ksamflk ksjfmk  [SEP]']


In [154]:
text = "ali ata bak"
text2 = "bak ali bak asd asdf aslşfg"

tokenizer([text, text2])["input_ids"]

[[3337, 5072, 2469],
 [2469, 3337, 2469, 2205, 1018, 2205, 18654, 5112, 1036, 1024, 1015]]

In [157]:
tokenizer(text, return_tensors="pt").to("cuda")

{'input_ids': tensor([[3337, 5072, 2469]], device='cuda:0'), 'token_type_ids': tensor([[0, 0, 0]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1]], device='cuda:0')}

In [176]:
import torch

# 3x4 boyutlu bir tensor oluşturalım
tensor = torch.randint(0, 5, (3, 4))

# Değeri 2 olan elemanların indekslerini bulalım
indices = torch.where(tensor == 2)

print("Tensor:\n", tensor)
print("Değeri 2 olan elemanların indeksleri (satır, sütun):", indices)

# Her satır için ilk 2'nin indeksi
row_mask = (tensor == 2).int()  # Boolean'ı int'e çevir
first_indices = row_mask.argmax(dim=1)  # Her satır için en yüksek değerin indeksi

# Eğer 2 yoksa -1 döner
first_indices[first_indices == 0] = -1  # Eğer 2 yoksa -1 atanır

print("Her satırdaki ilk 2'nin indeksi:", first_indices)


Tensor:
 tensor([[4, 4, 4, 1],
        [4, 0, 3, 2],
        [0, 0, 2, 0]])
Değeri 2 olan elemanların indeksleri (satır, sütun): (tensor([1, 2]), tensor([3, 2]))
Her satırdaki ilk 2'nin indeksi: tensor([-1,  3,  2])


In [193]:
tensor = torch.tensor([[1, 2, 3, 2],
                        [5, 2, 7, 8],
                        [9, 10, 2, 12]])

mask = (tensor == 2).int()

_, indices = mask.topk(2, dim=1)

indices[:, 0]


tensor([1, 1, 2])